Estiamte stages of delivery

In [1]:
### Set-ups
import sqlalchemy
from sqlalchemy import create_engine, inspect, select, text, and_, or_, func
from sqlalchemy import Table, MetaData

import pandas as pd

In [2]:
### Connecting to Database using my password
passwd = 'FyZSMAiH'
w_engine = create_engine('mysql+pymysql://e0376935:' + passwd + '@localhost:3306/olist', echo=True)

In [3]:
m2 = MetaData(bind=w_engine)
m2.reflect()

2022-03-16 22:08:28,131 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-16 22:08:28,131 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,170 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-03-16 22:08:28,170 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,189 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-03-16 22:08:28,189 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,248 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `olist`
2022-03-16 22:08:28,249 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,278 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `customers`
2022-03-16 22:08:28,279 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,300 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `geo_loc`
2022-03-16 22:08:28,301 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:28,322 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `order_items`
2022-03-16 22:08:28,323 INFO sql

In [4]:
for tt in m2.tables:
    print(tt)

customers
geo_loc
order_items
order_reviews
orders
payments
prod_cat_name
products
seller


Example of converting a quary result to a panda.dataframe

In [5]:
meas_table = Table("customers", m2, autoload=True)

# Select all columns of customers table
t1 = select([meas_table])

with w_engine.connect() as con2:
    d1 = con2.execute(t1)

s1 = pd.DataFrame.from_records(d1.fetchall())
type(s1)


2022-03-16 22:08:33,677 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.customer_uid, customers.zip_code, customers.city, customers.customer_state 
FROM customers
2022-03-16 22:08:33,677 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {}


pandas.core.frame.DataFrame

In [64]:
s1[3][0]

'osasco'

In [6]:
# Get dataframe for geo_avg

temp1 = Table("geo_avg", m2, autoload=True)

temp2 = select([temp1])

with w_engine.connect() as con2:
    temp3 = con2.execute(temp2)

temp4 = temp3.fetchall()

geo_avg = pd.DataFrame.from_records(temp4)
geo_avg.columns = temp4[0].keys()


2022-03-16 22:08:38,408 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `geo_avg`
2022-03-16 22:08:38,409 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:38,435 INFO sqlalchemy.engine.Engine DESCRIBE `geo_avg`
2022-03-16 22:08:38,436 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:08:38,500 INFO sqlalchemy.engine.Engine SELECT geo_avg.zip_code, geo_avg.latitude, geo_avg.longitude, geo_avg.city, geo_avg.state 
FROM geo_avg
2022-03-16 22:08:38,500 INFO sqlalchemy.engine.Engine [generated in 0.00058s] {}


In [7]:
# Get dataframe for time_table

temp1 = Table("time_table_6", m2, autoload=True)

temp2 = select([temp1])

with w_engine.connect() as con2:
    temp3 = con2.execute(temp2)

temp4 = temp3.fetchall()

time_table = pd.DataFrame.from_records(temp4)
time_table.columns = temp4[0].keys()


2022-03-16 22:09:02,008 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `time_table_6`
2022-03-16 22:09:02,009 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:09:02,030 INFO sqlalchemy.engine.Engine DESCRIBE `time_table_6`
2022-03-16 22:09:02,030 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-16 22:09:02,073 INFO sqlalchemy.engine.Engine SELECT time_table_6.customer_id, time_table_6.order_id, time_table_6.seller_id, time_table_6.approval_duration, time_table_6.carrier_duration, time_table_6.customer_duration, time_table_6.cust_lat, time_table_6.cust_long, time_table_6.seller_lat, time_table_6.seller_long, time_table_6.weight, time_table_6.length_cm, time_table_6.height_cm, time_table_6.width_cm, time_table_6.carrier_limit, time_table_6.estimated_delivery 
FROM time_table_6
2022-03-16 22:09:02,074 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {}


In [8]:
for i in geo_avg.columns:
    print(i)

zip_code
latitude
longitude
city
state


Build a regression model 
Parameter: distance and parcel weight
Response: approval duration (in second)

In [14]:
# Return distance between seller and customer
import geopy.distance

coords_1 = (time_table['cust_lat'][0],time_table['cust_long'][0])
coords_2 = (time_table['seller_lat'][0],time_table['seller_long'][0])

distance = geopy.distance.distance(coords_1, coords_2).km
distance

301.85895887168607

In [2]:
!pip install jaxlib

ERROR: Could not find a version that satisfies the requirement jaxlib (from versions: none)
ERROR: No matching distribution found for jaxlib


In [1]:
## To make the function runs faster, using jax here
from jax import vmap

def dis(coord1_1, coord1_2, coord2_1, coord2_2):
    coord1 = (coord1_1, coord1_2)
    coord2 = (coord2_1, coord2_2)
    return geopy.distance.distance(coords_1, coords_2).km


dis_vmap = vmap(dis, in_axes=(0,0,0,0))
distance = dis_vmap(time_table['cust_lat'][0:10],time_table['cust_long'][0:10],
 time_table['seller_lat'][0:10],time_table['seller_long'][0:10])

ModuleNotFoundError: No module named 'jaxlib'